# UKKO - Transformer model for tabular longitudonal data 

Author: Eric Fey (eric.fey@hus.fi)  
First version: 2025-03-01

This **`ukko_get_started`** notebook shows the basic cababilities of the ukko dual attention model:
- Predict next timepoints in timourse data. Example: sine wave.
- Classification based on sine wave feature data as example.


## Overview

UKKO is a PyTorch-based package for analyzing longitudinal tabular data using transformer-based deep learning models. It provides:

- **Dual Attention Architecture**: Combines feature-wise and temporal attention mechanisms
- **Multiple Model Types**: 
  - Regression models for time series prediction
  - Classification models for feature-wise classification
  - Survival analysis models based on Cox proportional hazards
- **Built-in Visualization**: Tools for model interpretation and performance analysis

### Installation

1. Install hatchling package manager:
```bash
pip install hatchling
```

2. Install ukko in development mode:
```bash
pip install -e .
```

3. Go :smiley:

### Quick Start

```python
import ukko
from ukko.core import DualAttentionRegressor
from ukko.data import SineWaveDataset

# Create synthetic dataset
dataset = SineWaveDataset(
    n_samples=1000,
    n_features=5,
    sequence_length=32,
    prediction_length=5
)

# Initialize model
model = DualAttentionRegressor(
    n_features=5,
    time_steps=32,
    d_model=128
)
```

### Contents

- TBA

### Requirements

- Python 3.8+
- PyTorch 2.0+
- NumPy
- Pandas
- Matplotlib
- Scikit-learn

For survival analysis:
- Lifelines
- Scikit-survival

### References

- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- [Deep Learning for Survival Analysis](https://arxiv.org/abs/1910.00199)

# Code Sections

In [ ]:
!pip install -e .

In [ ]:
# attention model with residual connectoin
import torch
import torch.nn as nn
import math
import ukko 
import importlib
import ukko.core
import ukko.data

importlib.reload(ukko.core)
importlib.reload(ukko.data)
importlib.reload(ukko.test)


In [ ]:
# Test ukko installatin and model
%run ./tests/tests_core.py
model = test_model()

## Test data sine wave

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from datetime import datetime
import ukko

def create_data_loaders(batch_size=32, n_samples=1000, n_features=15,
                       sequence_length=100, prediction_length=5):
    """Create train, validation, and test data loaders"""

    # Create datasets
    train_dataset = ukko.data.SineWaveDataset(n_samples, n_features, sequence_length, prediction_length)
    val_dataset = ukko.data.SineWaveDataset(n_samples//5, n_features, sequence_length, prediction_length, seed=43)
    test_dataset = ukko.data.SineWaveDataset(n_samples//5, n_features, sequence_length, prediction_length, seed=44)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader

def test_model_with_sine_data():
    # Parameters
    batch_size = 32
    n_samples = 1000
    n_features = 3
    sequence_length = 32
    prediction_length = 5

    # Create data loaders
    train_loader, val_loader, test_loader = create_data_loaders(
        batch_size=batch_size,
        n_samples=n_samples,
        n_features=n_features,
        sequence_length=sequence_length,
        prediction_length=prediction_length
    )

    # Initialize model
    model = ukko.core.DualAttentionRegressor(
        n_features=n_features,
        time_steps=sequence_length,
        d_model=128,
        n_heads=8,
        dropout=0.1
    )
    #model = []


    # Plot example
    dataset = ukko.data.SineWaveDataset(n_samples, n_features, sequence_length, prediction_length)
    ukko.data.plot_example_dataset(dataset, sample_idx=1)

    return model, train_loader, val_loader, test_loader

#if __name__ == "__main__":
model, train_loader, val_loader, test_loader = test_model_with_sine_data()

In [ ]:
importlib.reload(ukko.data)

# Parameters
batch_size = 32
n_samples = 1400
n_features = 5
sequence_length = 32
prediction_length = 5

# Create dataset:
dataset = ukko.data.SineWaveDataset(n_samples, n_features, sequence_length, prediction_length,
                                   base_freq=0.05, noise_level=0.2)
ukko.data.plot_example_dataset(dataset, sample_idx=0, feature_idx=0)
ukko.data.plot_example_dataset(dataset, sample_idx=1, feature_idx=0)
ukko.data.plot_example_dataset(dataset, sample_idx=0, feature_idx=1)
ukko.data.plot_example_dataset(dataset, sample_idx=1, feature_idx=1)
ukko.data.plot_example_dataset(dataset, sample_idx=0, feature_idx=2)
ukko.data.plot_example_dataset(dataset, sample_idx=1, feature_idx=2)

trainset = dataset.Subset(range(0,1000))
valset = dataset.Subset(range(1000,1200))
testset = dataset.Subset(range(1200,1400))

print(f"Trainset: {trainset.__len__()}")
print(f"Trainset: {trainset.getdim()}")
print(f"Valset:   {valset.getdim()}")
print(f"Testset:  {testset.getdim()}")

# Create data loaders
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size)
test_loader = DataLoader(testset, batch_size=batch_size)

In [ ]:
# Initialize model
model = ukko.core.DualAttentionRegressor(
#model = ukko.core.DualAttentionModelOld(
    n_features=n_features,
    time_steps=sequence_length,
    d_model=128,
    n_heads=16,
    dropout=0.1
)

In [ ]:
from itertools import islice
for x, y in islice(val_loader, 2):
  print(x.shape)
  print(y.shape)

In [ ]:
def train_sine_model(model, train_loader, val_loader, epochs=50, lr=0.001, device='cuda'):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            output, _, _ = model(x)
            #loss = criterion(output, y[:, :, 0])  # Compare with first time step of prediction
            loss = criterion(output, y)  # y is only one timepoint 5 or 4 steps ahead

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                output, _, _ = model(x)
                #val_loss += criterion(output, y[:, :, 0]).item()
                val_loss += criterion(output, y).item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_sine_model.pt')

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model, train_loader, val_loader, test_loader = test_model_with_sine_data()
train_sine_model(model, train_loader, val_loader, epochs = 50, device = device)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

importlib.reload(ukko.data)

def visualize_predictions(model, test_loader, device='cuda', num_examples=3):
    model.eval()

    # Get some test examples
    x, y = next(iter(test_loader))
    x, y = x.to(device), y.to(device)

    # Get predictions
    with torch.no_grad():
        predictions, feat_attn, time_attn = model(x)
        predictions = predictions.cpu().numpy()
        x = x.cpu().numpy()
        y = y.cpu().numpy()

    # Plot 3 different features
    for i, feature_idx in enumerate([0,1,2]):#([0, 7, 14]):  # Beginning, middle, and end features
        
        # Plot multiple features for a few examples
        for example_idx in range(min(num_examples, x.shape[0])):
            
            fig = ukko.data.plot_example_dataset(testset, example_idx, feature_idx)
            ax = fig.axes[0]
            
            # Time of prediction 
            time_input = np.arange(x.shape[2])
            #time_target = np.arange(x.shape[2], x.shape[2] + y.shape[2])
            time_target = [x.shape[2] + prediction_length - 1]
            
            # Plot prediction
            plt.figure(figsize=(15, 10))
            ax.scatter(time_target[0], predictions[example_idx, feature_idx].item(),
                       marker = '^', label='Model Prediction', color='red', s=100, 
                      zorder=5)
            
            ax.set_title(f'Example {example_idx}, Feature {feature_idx}')
            ax.legend()
            ax.grid(True)
        
    # Plot data vs predicted & groundtruth vs predicted
    plt.figure(figsize=(15, 5))
    # data vs predicted
    plt.subplot(1, 2, 1)
    print(f"Data: {y.shape}")
    print(f"Predictions: {predictions.shape}")
    np.random.seed(1)
    for col in range(y.shape[1]):
      plt.scatter(predictions[:,col], y[:,col],
                  c=np.random.rand(3,), label=f'Feature {col}', alpha=0.5)
    # Add perfect line
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Data vs Predicted')
    plt.legend()

    # Visualize attention weights
    plt.figure(figsize=(15, 5))

    # Feature attention weights
    plt.subplot(1, 2, 1)
    feat_attn_avg = feat_attn.mean(dim=(0, 1)).cpu().numpy()
    plt.imshow(feat_attn_avg, aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.title('Feature Attention Weights')
    plt.xlabel('Target Feature')
    plt.ylabel('Source Feature')

    # Time attention weights
    plt.subplot(1, 2, 2)
    time_attn_avg = time_attn.mean(dim=(0, 1)).cpu().numpy()
    plt.imshow(time_attn_avg, aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.title('Time Attention Weights')
    plt.xlabel('Target Time Step')
    plt.ylabel('Source Time Step')

    plt.tight_layout()
    plt.show()

def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    criterion = torch.nn.MSELoss()
    total_mse = 0
    total_mae = 0
    num_batches = 0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            predictions, _, _ = model(x)

            # Calculate MSE and MAE
            #mse = criterion(predictions, y[:, :, 0])
            #mae = torch.abs(predictions - y[:, :, 0]).mean()
            mse = criterion(predictions, y)
            mae = torch.abs(predictions - y).mean()

            total_mse += mse.item()
            total_mae += mae.item()
            num_batches += 1

    avg_mse = total_mse / num_batches
    avg_mae = total_mae / num_batches

    print(f"Test Set Metrics:")
    print(f"Average MSE: {avg_mse:.4f}")
    print(f"Average MAE: {avg_mae:.4f}")
    print(f"RMSE: {np.sqrt(avg_mse):.4f}")

    return avg_mse, avg_mae

def main():
    # Parameters
    # batch_size = 32
    # n_samples = 1000
    # n_features = 3
    # sequence_length = 32
    # prediction_length = 5

    # Create data loaders
    # train_loader, val_loader, test_loader = create_data_loaders(
    #     batch_size=batch_size,
    #     n_samples=n_samples,
    #     n_features=n_features,
    #     sequence_length=sequence_length,
    #     prediction_length=prediction_length
    # )

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model = ukko.core.DualAttentionModel(
    #     n_features=n_features,
    #     time_steps=sequence_length,
    #     d_model=128,
    #     n_heads=8,
    #     dropout=0.1
    # ).to(device)

    # Load best model weights
    try:
        model.load_state_dict(torch.load('best_sine_model.pt'))
        print("Loaded best model weights successfully!")
    except:
        print("No saved model found. Please train the model first.")
        return

    # Evaluate model
    mse, mae = evaluate_model(model, test_loader, device)

    # Visualize results
    visualize_predictions(model, test_loader, device)

    # Save a timestamp of when these results were generated
    timestamp = "2025-03-01 10:01:16"  # Using the provided timestamp
    user = "eric-fey-hus"

    # Save results to a file
    with open('model_results.txt', 'w') as f:
        f.write(f"Results generated on: {timestamp}\n")
        f.write(f"Generated by user: {user}\n")
        f.write(f"Model Performance Metrics:\n")
        f.write(f"MSE: {mse:.4f}\n")
        f.write(f"MAE: {mae:.4f}\n")
        f.write(f"RMSE: {np.sqrt(mse):.4f}\n")

#if __name__ == "__main__":
main()

# Classifier

In [ ]:
# First reload modules to ensure latest changes
import importlib
import ukko.core
import ukko.data
importlib.reload(ukko.core)
importlib.reload(ukko.data)

# Import test function 
from ukko.core import DualAttentionClassifier, visualize_predictions
from ukko.data import SineWaveDataset

# Run the test
#from tests.test_dual_attention_classifier import test_dual_attention_classifier
#test_dual_attention_classifier()

%run ./tests/test_dual_attention_classifier.py
model, train_losses, plt1, plt2 = test_dual_attention_classifier(n_epochs = 3)